# Prepair GANDALF Jr.

In [1]:
import os, cv2, glob, datetime, sys, glob
from PIL import Image
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow, imsave
%matplotlib inline

# import gandalf tools
from gandalfs_tools import *

In [12]:
step, starting_epoch, starting_step, inum, n_noise, n_critic = 1, 0, 0, 100, 100, 1
ida, sessionid = "", "1"

IMAGE_DIM, normval = (128, 128, 3), (0.5, 0.5, 0.5)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# set local paths
SAVE_DIR_ROOT = 'D:\\GANDALFSWELT\\out\\'
mempath_root = 'D:\\GANDALFSWELT\\checkpoints\\'

INPUT_IMAGES ="D:\\GANDALFSWELT\\input\\back_in\\"
RESIZED_IMAGES = "D:\\GANDALFSWELT\\input\\resized\\"

READY_IMAGES = "D:\\GANDALFSWELT\\input\\ready\\"
READYPLUS_IMAGES =  "D:\\GANDALFSWELT\\input\\readyplus\\"
ORG_IMAGES = "D:\\GANDALFSWELT\\input\\readyorg\\"
GANDALFS_IMAGES = "D:\\GANDALFSWELT\\input\\backinall\\"

transform = transforms.Compose([transforms.Resize((IMAGE_DIM[0],IMAGE_DIM[1])),transforms.ToTensor(),
                                transforms.Normalize(mean=normval,std=normval)])

In [13]:
# get last memory path
mempath = mempath_root + str(max([int(x.replace('.pt','')) for x in os.listdir(mempath_root)])) + '.pt'
print(mempath)

# set run number
rnum = max([int(x.replace('.pt','')) for x in os.listdir(mempath_root)])
print("\t\t\t\t...current rnum:",rnum)

D:\GANDALFSWELT\checkpoints\1889.pt
				...current rnum: 1889


# Create GANDALF Jr.

In [14]:
D = Discriminator(in_channel=IMAGE_DIM[-1]).to(DEVICE)
G = Generator(out_channel=IMAGE_DIM[-1]).to(DEVICE)
criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters(), lr=0.001, betas=(0.5, 0.999)) # lr = 0.001
G_opt = torch.optim.Adam(G.parameters(), lr=0.001, betas=(0.5, 0.999))
G.eval()

Generator(
  (fc): Sequential(
    (0): Linear(in_features=100, out_features=32768, bias=True)
    (1): ReLU()
  )
  (conv): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): ConvTranspose2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): ConvTranspose2d(128, 3, kernel_size=(4, 4), 

# Reload GANDALF Jr. from memory

In [15]:
checkpoint = torch.load(mempath)
D.load_state_dict(checkpoint['D'])
G.load_state_dict(checkpoint['G'])
D_opt.load_state_dict(checkpoint['d_optim'])
G_opt.load_state_dict(checkpoint['g_optim'])        
G.train()
D.train()

Discriminator(
  (conv): Sequential(
    (0): Conv2d(3, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Conv2d(512, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Conv2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.2)
    (12): AdaptiveAvgPool2d(output_size=1)
  )
  (fc): Sequential(
    (0): Linear(in_featu

# Input Images / Preprocessing

#### batch size max.: 89

In [16]:
batch_size = 89

In [29]:
# preprocessing step 1: resize
preprocess_step1([INPUT_IMAGES+x for x in os.listdir(INPUT_IMAGES)], RESIZED_IMAGES)

# preprocessing step 2: convert and save
preprocess_step2(READY_IMAGES, RESIZED_IMAGES)
preprocess_step2(GANDALFS_IMAGES, RESIZED_IMAGES)
preprocess_step2(READYPLUS_IMAGES, RESIZED_IMAGES)

In [30]:
def pic_num(path):
    return len(os.listdir(path))

A_p, O_p, G_p, Ap_p = pic_num(READY_IMAGES), pic_num(ORG_IMAGES), pic_num(GANDALFS_IMAGES), pic_num(READYPLUS_IMAGES)
Ap_ev, A_ev, O_ev = int(Ap_p/batch_size), int(A_p/batch_size), int(O_p/batch_size)

print('Num Original Pics:\t',O_p, '\t Num Batches:', round((O_p/batch_size),2),
      '\nNum Generated Pics:\t',G_p,'\t Num Batches:', round((G_p/batch_size),2),
      '\n\nNum Total Input Pics:\t',A_p,'\t Num Batches:', round((A_p/batch_size),2),
      '\nNum Input Pics Plus:\t',Ap_p,'\t Num Batches:', round((Ap_p/batch_size),2))

Num Original Pics:	 2484 	 Num Batches: 27.91 
Num Generated Pics:	 2541 	 Num Batches: 28.55 

Num Total Input Pics:	 4551 	 Num Batches: 51.13 
Num Input Pics Plus:	 5024 	 Num Batches: 56.45


# GANDALF JR starts creating

In [69]:
# settings for output pics & folders:

# NOTES:
# scale_ev 0.5 gut für video

update_savedir = True
reset_stepnum = False

mixes = 1
scale_epoch_num = 1  
scale_ev = 0.5

## Set No. 1: All Plus Pics
**Original and Gandalf Input Pics plus small selection of horizontally flipped G pics**

In [70]:
max_epoch = int(scale_ev*10*scale_epoch_num) 
ev = int(Ap_ev*scale_ev) 
print("num epochs per mix:", max_epoch, "\t pic ev:", ev)

if update_savedir:
    SAVE_DIR = SAVE_DIR_ROOT+str(rnum+1)+"_scev"+ str(int(scale_ev*100))+"_Ap" +"\\"
    makesavepath(SAVE_DIR)
    
if reset_stepnum:
    step = 0

num epochs per mix: 5 	 pic ev: 28
made folder D:\GANDALFSWELT\out\kater\1898_scev50_Ap\


In [71]:
data_path = READYPLUS_IMAGES
dataset = IMAGES(data_path=data_path,transform=transform)

In [ ]:
inum = 100000
rnum += 1

for mix in range(mixes):
    
    print("Mix",mix+1,"of",mixes)
    data_loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,drop_last=True,num_workers=0)
    D_labels = torch.ones([batch_size, 1]).to(DEVICE)                       # Discriminator Label to real
    D_fakes = torch.zeros([batch_size, 1]).to(DEVICE)                       # Discriminator Label to fake
    
    for epoch in range(max_epoch):
        for idx, images in enumerate(data_loader):                          # D  
            x = images.to(DEVICE)
            x_outputs = D(x)
            D_x_loss = criterion(x_outputs, D_labels)
            z = torch.randn(batch_size, n_noise).to(DEVICE)                 # G -> D
            z_outputs = D(G(z))
            D_z_loss = criterion(z_outputs, D_fakes)
            D_loss = D_x_loss + D_z_loss
            D.zero_grad()                                                   # D loss + opt
            D_loss.backward()
            D_opt.step()
            if step % n_critic == 0:                                        # G loss + opt
                z = torch.randn(batch_size, n_noise).to(DEVICE)
                z_outputs = D(G(z))
                G_loss = criterion(z_outputs, D_labels)
                D.zero_grad()
                G.zero_grad()
                G_loss.backward()
                G_opt.step()
            if step % ev == 0:                                              # get pic
                name = SAVE_DIR +ida+str(rnum)+ "_"+str(inum)[1:]+ '.jpg'
                print('\n\tEpoch: {}/{}, D Loss: {:.4f}, G Loss: {:.4f}'.format(epoch+1,max_epoch,D_loss.item(),G_loss.item()))
                G.eval()
                img = generate_image(G, 1, n_noise)
                imsave(name, img[0],dpi=600)
                print("\t"*5,"saving",name)
                inum +=1
                G.train()
            step += 1       

Mix 1 of 1

	Epoch: 1/5, D Loss: 0.0018, G Loss: 7.7924
					 saving D:\GANDALFSWELT\out\kater\1898_scev50_Ap\1898_00000.jpg

	Epoch: 1/5, D Loss: 0.0014, G Loss: 6.9597
					 saving D:\GANDALFSWELT\out\kater\1898_scev50_Ap\1898_00001.jpg

	Epoch: 2/5, D Loss: 0.0021, G Loss: 7.6317
					 saving D:\GANDALFSWELT\out\kater\1898_scev50_Ap\1898_00002.jpg

	Epoch: 2/5, D Loss: 0.0029, G Loss: 6.4516
					 saving D:\GANDALFSWELT\out\kater\1898_scev50_Ap\1898_00003.jpg


In [ ]:
mempath = mempath_root + str(rnum) + ".pt"
torch.save({'D':D.state_dict(),'G':G.state_dict(),'d_optim':D_opt.state_dict(),'g_optim':G_opt.state_dict()},mempath) 
mempath

## Set No. 2: All Pics
**Original and Gandalf Input Pics**

In [ ]:
max_epoch = int(scale_ev*10*scale_epoch_num) 
ev = int(Ap_ev*scale_ev) 
print("num epochs per mix:", max_epoch, "\t pic ev:", ev)

if update_savedir:
    SAVE_DIR = SAVE_DIR_ROOT+str(rnum+1)+"_scev"+ str(int(scale_ev*100))+"_Ap" +"\\"
    makesavepath(SAVE_DIR)
    
if reset_stepnum:
    step = 0

In [ ]:
data_path = READY_IMAGES
dataset = IMAGES(data_path=data_path,transform=transform)

In [ ]:
inum = 100000
rnum += 1

for mix in range(mixes):
    
    print("Mix",mix+1,"of",mixes)
    data_loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,drop_last=True,num_workers=0)
    D_labels = torch.ones([batch_size, 1]).to(DEVICE)                       # Discriminator Label to real
    D_fakes = torch.zeros([batch_size, 1]).to(DEVICE)                       # Discriminator Label to fake
    
    for epoch in range(max_epoch):
        for idx, images in enumerate(data_loader):                          # D  
            x = images.to(DEVICE)
            x_outputs = D(x)
            D_x_loss = criterion(x_outputs, D_labels)
            z = torch.randn(batch_size, n_noise).to(DEVICE)                 # G -> D
            z_outputs = D(G(z))
            D_z_loss = criterion(z_outputs, D_fakes)
            D_loss = D_x_loss + D_z_loss
            D.zero_grad()                                                   # D loss + opt
            D_loss.backward()
            D_opt.step()
            if step % n_critic == 0:                                        # G loss + opt
                z = torch.randn(batch_size, n_noise).to(DEVICE)
                z_outputs = D(G(z))
                G_loss = criterion(z_outputs, D_labels)
                D.zero_grad()
                G.zero_grad()
                G_loss.backward()
                G_opt.step()
            if step % ev == 0:                                              # get pic
                name = SAVE_DIR +ida+str(rnum)+ "_"+str(inum)[1:]+ '.jpg'
                print('\n\tEpoch: {}/{}, D Loss: {:.4f}, G Loss: {:.4f}'.format(epoch+1,max_epoch,D_loss.item(),G_loss.item()))
                G.eval()
                img = generate_image(G, 1, n_noise)
                imsave(name, img[0],dpi=600)
                print("\t"*5,"saving",name)
                inum +=1
                G.train()
            step += 1       

In [ ]:
mempath = mempath_root + str(rnum) + ".pt"
torch.save({'D':D.state_dict(),'G':G.state_dict(),'d_optim':D_opt.state_dict(),'g_optim':G_opt.state_dict()},mempath) 
mempath

## Set No. 3) Original Pis 
**Only Original Input Pics**

In [ ]:
max_epoch = int(scale_ev*10*scale_epoch_num) * int((Ap_p/batch_size)/(O_p/batch_size)) # multiply with factorization O/A
ev = int(Ap_ev*scale_ev) 
print("num epochs per mix:", max_epoch, "\t pic ev:", ev)

if update_savedir:
    SAVE_DIR = SAVE_DIR_ROOT+str(rnum+1)+"_scev"+ str(int(scale_ev*100))+"_Ap" +"\\"
    makesavepath(SAVE_DIR)
    
if reset_stepnum:
    step = 0

In [ ]:
data_path = ORG_IMAGES
dataset = IMAGES(data_path=data_path,transform=transform)

In [ ]:
inum = 100000
rnum += 1

for mix in range(mixes):
    
    print("Mix",mix+1,"of",mixes)
    data_loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,drop_last=True,num_workers=0)
    D_labels = torch.ones([batch_size, 1]).to(DEVICE)                       # Discriminator Label to real
    D_fakes = torch.zeros([batch_size, 1]).to(DEVICE)                       # Discriminator Label to fake
    
    for epoch in range(max_epoch):
        for idx, images in enumerate(data_loader):                          # D  
            x = images.to(DEVICE)
            x_outputs = D(x)
            D_x_loss = criterion(x_outputs, D_labels)
            z = torch.randn(batch_size, n_noise).to(DEVICE)                 # G -> D
            z_outputs = D(G(z))
            D_z_loss = criterion(z_outputs, D_fakes)
            D_loss = D_x_loss + D_z_loss
            D.zero_grad()                                                   # D loss + opt
            D_loss.backward()
            D_opt.step()
            if step % n_critic == 0:                                        # G loss + opt
                z = torch.randn(batch_size, n_noise).to(DEVICE)
                z_outputs = D(G(z))
                G_loss = criterion(z_outputs, D_labels)
                D.zero_grad()
                G.zero_grad()
                G_loss.backward()
                G_opt.step()
            if step % ev == 0:                                              # get pic
                name = SAVE_DIR +ida+str(rnum)+ "_"+str(inum)[1:]+ '.jpg'
                print('\n\tEpoch: {}/{}, D Loss: {:.4f}, G Loss: {:.4f}'.format(epoch+1,max_epoch,D_loss.item(),G_loss.item()))
                G.eval()
                img = generate_image(G, 1, n_noise)
                imsave(name, img[0],dpi=600)
                print("\t"*5,"saving",name)
                inum +=1
                G.train()
            step += 1       

In [ ]:
mempath = mempath_root + str(rnum) + ".pt"
torch.save({'D':D.state_dict(),'G':G.state_dict(),'d_optim':D_opt.state_dict(),'g_optim':G_opt.state_dict()},mempath)
mempath